<a href="https://colab.research.google.com/github/KangTuna/Machine-learning-and-deep-learning-practices/blob/main/GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math # 수학 패키지
import torch # 파이토치 모듈
import matplotlib.pyplot as plt # 시각화
import numpy as np # 넘파이
import pandas as pd # 판다스
import torch.nn as nn
import torch.optim as optim

In [ ]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
import torchvision.datasets as datasets


In [ ]:
# 디바이스 할당
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,)),
])

In [ ]:
train_dataset = datasets.MNIST(
    root= './data', train=True,transform=transform, download = True
)

100%|██████████| 9912422/9912422 [00:00<00:00, 43555253.98it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 99373005.60it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 11616474.62it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 23519171.32it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
batch_size = 512

In [ ]:
train_loader = DataLoader(
    train_dataset, batch_size = batch_size, shuffle= True
)

In [ ]:
nz = 128

In [ ]:
class Generator(nn.Module):
  def __init__(self,nz):
    super(Generator,self).__init__()
    self.nz = nz
    self.main = nn.Sequential(
       nn.Linear(self.nz,256),
       nn.LeakyReLU(0.2) ,
       nn.Linear(256,512),
       nn.LeakyReLU(0.2) ,
       nn.Linear(512,1024),
       nn.LeakyReLU(0.2) ,
       nn.Linear(1024,784),
       nn.Tanh(),
    )
  def forward(self,x):
    return self.main(x).view(-1,1,28,28)

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator,self).__init__()
    self.n_input = 784
    self.main = nn.Sequential(
        nn.Linear(self.n_input,1024),
        nn.LeakyReLU(0.2) ,
        nn.Dropout(0.3),
        nn.Linear(1024,512),
        nn.LeakyReLU(0.2) ,
        nn.Dropout(0.3),
        nn.Linear(512,256),
        nn.LeakyReLU(0.2) ,
        nn.Dropout(0.3),
        nn.Linear(256,1),
        nn.Sigmoid(),
    )
  def forward(self, x):
    x = x.view(-1,784)
    return self.main(x)


In [ ]:
generator = Generator(nz).to(device)
discriminator = Discriminator().to(device)
print(generator)
print(discriminator)

In [ ]:
optim_g = optim.Adam(generator.parameters(), lr = 0.0002)
optim_d = optim.Adam(discriminator.parameters(), lr = 0.0002)

criterion = nn.BCELoss()

losses_g = []
losses_d = []
images = []

In [ ]:
from torchvision.utils import make_grid

In [ ]:
def train_discriminator(optimizer,data_real,data_fake):
  b_size = data_real.size(0)
  real_label = torch.ones(b_size,1).to(device)
  fake_label = torch.zeros(b_size,1).to(device)
  optimizer.zero_grad()
  output_real = discriminator(data_real)
  loss_real = criterion(output_real, real_label)
  output_fake = discriminator(data_fake)
  loss_fake = criterion(output_fake, fake_label)
  loss_total = (loss_real + loss_fake)/2
  loss_total.backward()
  optimizer.step()
  return loss_total

def train_generator(optimizer,data_fake):
  b_size = data_fake.size(0)
  real_label = torch.ones(b_size, 1).to(device)
  optimizer.zero_grad()
  output = discriminator(data_fake)
  loss = criterion(output, real_label)
  loss.backward()
  optimizer.step()
  return loss

In [ ]:
epochs = 10
for epoch in range(epochs):
  loss_g = 0.0
  loss_d = 0.0
  for idx,data in enumerate(train_loader):
    image, _= data
    data_real = image.to(device)
    b_size = len(data_real)

    data_fake = generator(torch.randn(b_size,nz).to(device))

    loss_d += train_discriminator(optim_d, data_real,data_fake)

    loss_g += train_generator(optim_g, data_fake)

    generator_img = generator(torch.randn(b_size,nz).to(device).cpu().detach())
    generator_img = make_grid(generator_img)

    images.append(generator_img)
    epoch_loss_g = loss_g / idx
    epoch_loss_d = loss_d / idx
    losses_g.append(epoch_loss_g)
    losses_d.append(epoch_loss_d)

  print(f"Epoch {epoch} of {epochs}")
  print(f'Generator loss : {epoch_loss_g:.8f}, Discriminator loss {epoch_loss_d:.8f}')

RuntimeError: ignored

In [ ]:
data_fake = generator(torch.randn(b_size,nz).to(device))

In [ ]:
# data_fake = generator(torch.randn(b_size,nz).to(device))
# loss_g += train_generator(optim_g, data_fake)

# loss_d += train_discriminator(optim_d, data_real,data_fake)

NotImplementedError: ignored

In [ ]:
generator_img = generator(torch.randn(b_size,nz).to(device).cpu().detach())
generator_img = make_grid(generator_img)

images.append(generator_img)
epoch_loss_g = loss_g / idx
epoch_loss_d = loss_d / idx
losses_g.append(epoch_loss_g)
losses_d.append(epoch_loss_d)

print(f"Epoch {epoch} of {epochs}")
print(f'Generator loss : {epoch_loss_g:.8f}, Discriminator loss {epoch_loss_d:.8f}')